해당 코드는 저장 공간이 부족하기 때문에 폴더 및 파일 잘 확인해서 사용할 것.

--> aihub 인도보행영상 데이터 셋을 다운 받아 train /val 디렉토리에 넣으면 각 이미지에 맞는 바운딩 박스 라벨링을 txt파일로 바꿔준다(xml형식 어노테이션을 yolov5에 맞게 txt변환)

In [2]:
import xml.etree.ElementTree as elemTree
import os
import torch
import numpy as np
import cv2 
from glob import glob


#class names: ['person', 'bicycle', 'scooter', 'bollard', 'traffic_light']

pascal_class_index = {
    'person':0,
    'bicycle': 1,
    'scooter': 2,
    'bollard': 0,
    'traffic_light': 4
        
}


#train/ val/ test 디렉토리 변경
root_dir = "./train/"
img_dir = os.path.join(root_dir, 'images')  
xml_dir = os.path.join(root_dir, 'annotations')
label_dir = os.path.join(root_dir, 'labels')


def Read_xml():
    width = float(1920)
    height = float(1080)
    image_path = os.path.join(root_dir, 'images')
    dirs = glob(os.path.join(image_path,'*/*'))
    xmls = glob(os.path.join(image_path,'*/*/*.xml'))
    total_result = []
    #for xml in xmls:
    for sub_dir in dirs:
        xml_path = glob(os.path.join(sub_dir,"*.xml"))[0]
        tree = elemTree.parse(xml_path)
        for image in tree.findall('./image'):
            fname = image.attrib['name']
            box_result = []
            for box in image.findall("./box"):
                name = box.attrib["label"]
                try:
                    class_index = pascal_class_index[name]
                except KeyError:
                    continue
                
                #xtl,ytl이 좌상단/ xbr,ybr이 우하단 
                xmin = float(box.attrib["xtl"])
                ymin = float(box.attrib["ytl"])
                xmax = float(box.attrib["xbr"])
                ymax = float(box.attrib["ybr"])
                bb_width = round((xmax - xmin) / width, 15)
                bb_height = round((ymax - ymin) / height, 15)
                x_center = round((xmax + xmin)/2/width, 15)
                y_center = round((ymax + ymin)/2/height, 15)
                box_result.append([class_index, x_center, y_center, bb_width, bb_height])
            Write_txt(file_name = fname, result= box_result)

            
def Write_txt(file_name, result):
    file_name = file_name[:-3]+'txt'
    file_path = os.path.join(label_dir, file_name)
    f = open(file_path, 'w')
    for i, data in enumerate(result):
        data = f'{data}\n'
        data = data.replace(",", "").replace("[", "").replace("]", "")
        f.write(data)
    f.close()

 
def createfolder(dir):
    try:
        if not os.path.exists(dir):
            os.makedirs(dir)
    except OSError:
        print("Error : creating directory." + dir)
        


def main():
    if not os.path.isdir(xml_dir):
        raise Exception("there is no XML directory(for annotations)")
    createfolder(label_dir)
    Read_xml()

if __name__ == '__main__':
  main()
